In [5]:
# pip install tavily-python

import os
import json
import time
from openai import OpenAI
from tavily import TavilyClient
from IPython.display import display, Markdown

# Initialize clients with API keys
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

assistant_prompt_instruction = """You are an immigration expert having expertise in Finnish immigration and integration. 
Your goal is to provide updated information from the internet using Tavily search API function. 
If no relevant information is found on the internet, utilize your own knowledge
to answer the question. In either case, acknowledge the source of the information. 
If the information is not present in your own knowledge, simply say that you cannot provide the requested information.
In case of an internet search, Please include relevant url sources at the end of your answers.
Analyze the query to determine whether the question requires internet access to find the updated information. 
Internet access may be required for queries asking for information on rapidly changing policies, current statistics, or other recent facts (e.g., current tax rate, tution fees, visa processing fee, residency period, etc.
It is important to note that you only have to answer queries related to Finland in general (e.g, location, history, economy, etc.), Finnish immigration,
Finnish culture, Finnish lifestyle, or integration in Finland. For all other non-related queries, simply respond by giving an appropriate message
"""

# Create an assistant
assistant = client.beta.assistants.create(
    instructions=assistant_prompt_instruction,
    model="gpt-4o-mini",
    tools=[{
        "type": "function",
        "function": {
            "name": "tavily_search",
            "description": "Get information on recent events from the web.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query to use. For example: 'What is the average processing time of residence permit?'"},
                },
                "required": ["query"]
            }
        }
    }]
)
assistant_id = assistant.id
print(f"Assistant Created. Assistant ID: {assistant_id}")

Assistant Created. Assistant ID: asst_WL5Ttz9FaPKVit50Y8CaKaUm


In [ ]:
"""Create a thread for the assistant and maintain chat."""
import os
import json
import time
from openai import OpenAI
from tavily import TavilyClient
from IPython.display import display, Markdown

# Function to perform a Tavily search
def tavily_search(query):
    display(Markdown("Retrieving updated information..."))  # Status message when using TavilyClient
    search_result = tavily_client.get_search_context(query, search_depth="advanced", max_tokens=8000)
    return search_result

# Function to wait for a run to complete
def wait_for_run_completion(thread_id, run_id):
    while True:
        display(Markdown("Thinking..."))
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

# Function to handle tool output submission
def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "tavily_search":
            output = tavily_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

# Function to print messages from a thread, ensuring they appear in the correct order
def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    sorted_messages = sorted(messages, key=lambda msg: msg.created_at)  # Sort by creation time
    for msg in sorted_messages:
        display(Markdown(f"{msg.role}: {msg.content[0].text.value}"))

# Create a thread
thread = client.beta.threads.create()
print(f"Thread: {thread}")


# Ongoing conversation loop
while True:
    display(Markdown("**You (type 'exit' to end):**"))
    user_input = input()
    if user_input.lower() == 'exit':
        break

    # Create a message
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input,
    )

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

    # Wait for run to complete
    run = wait_for_run_completion(thread.id, run.id)

    if run.status == 'failed':
        print(run.error)
        continue
    elif run.status == 'requires_action':
        run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
        run = wait_for_run_completion(thread.id, run.id)

    # Print messages from the thread
    print_messages_from_thread(thread.id)


Thread: Thread(id='thread_YgRoszPFmcmHwa2OA5M1rah3', created_at=1727907293, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


**You (type 'exit' to end):**

 What is the current residency period for citizenship?


Thinking...

Retrieving updated information...

...

user: During the entire continuous period of residence, how much time the applicant may stay abroad.

assistant: During the entire continuous period of residence required for Finnish citizenship, an applicant may stay abroad for a total of **one year**. However, no more than **three months** of this period can be taken during the **year preceding** naturalization. This regulation aims to ensure that applicants are genuinely residing in Finland.

Sources for this information include:

1. [Valtioneuvosto - Stricter requirements for acquiring Finnish citizenship](https://valtioneuvosto.fi/en/-/1410869/stricter-requirements-for-acquiring-finnish-citizenship-the-required-period-of-residence-to-be-extended)
2. [Migri - Changes to residence period for citizenship](https://migri.fi/-/kansalaisuushakemusten-vaadittu-asumisaika-pitenee?languageId=en_US)

user: What will be the change in the tuition fees from 2024?

assistant: Starting in 2024, changes to tuition fees for non-European Union (EU) and non-European Economic Area (EEA) students in Finland are expected to come into effect. The Finnish government plans to implement full-cost tuition fees that cover the complete expenses associated with students' education. This reform is part of a broader strategy to enhance the funding of Finnish universities.

Key points regarding the changes include:

- Tuition fees will need to fully cover the costs of completing a degree for non-EU/EEA students.
- While specific fee amounts haven't been detailed in the sources, it's stated that existing tuition fees already cover a significant portion of these costs in institutions like the University of Helsinki.
- Additionally, the number of scholarships available for international students may be reduced as a result of these changes.

For more details, you can check the following sources:

1. [University World News - Proposal for full-cost tuition fees](https://www.universityworldnews.com/post.php?story=20240529081922658)
2. [YLE - Law changes on tuition fees for non-EU students](https://yle.fi/a/74-20089083)
3. [Daily Finland - Government moves to increase tuition fees](https://www.dailyfinland.fi/education/33771/Govt-moves-to-increase-tuition-fees-for-non-EU-non-EEA-students)

**You (type 'exit' to end):**